# Import Modules

In [1]:
import os
import pandas as pd
from credentials import *
import snowflake.connector
from Queries.TPCH_SF1_ORDERS import *

c:\Users\ASANTE\miniconda3\envs\SNOWFLAKE\lib\site-packages\snowflake\connector\options.py:108: UserWarning: You have an incompatible version of 'pyarrow' installed (12.0.1), please install a version that adheres to: 'pyarrow<10.1.0,>=10.0.1; extra == "pandas"'
  warn_incompatible_dep(
Failed to import ArrowResult. No Apache Arrow result set format can be used. ImportError: DLL load failed while importing arrow_iterator: The specified procedure could not be found.


# Helper Functions

In [2]:
def query_cursor(wh_name: str, db_name: str, schema_name: str):
    '''
    Create a cursor for interacting with database

    Parameters

    wh_name: str
      warehouse name
    
    db_name: str
      database name

    schema_name: str
      schema name

    Returns:
    The cursor itself, or None if some error happened, or the response returned by Snowflake if the _no_results flag is on.
    
    '''
    cnx = snowflake.connector.connect(user=USERNAME, password=PASSWORD, account=ACCOUNT_ID)

    cur = cnx.cursor().execute(f'USE WAREHOUSE {wh_name}')
    cur = cnx.cursor().execute(f'USE DATABASE {db_name}')
    cur = cnx.cursor().execute(f'USE SCHEMA {schema_name}')

    return cur


def fetch_data(wh_name, db_name, schema_name, sql_query, col_names) -> pd.DataFrame:
    '''
    Retrieves data from a Snowflake database and returns it as a Pandas DataFrame.

    Parameters

    wh_name: str
      warehouse name
    
    db_name: str
      database name

    schema_name: str
      schema name

    sql_query: str
        query to execute

    col_names: list[str]
        column names to use

    Returns:
        result as pandas dataframe
    
    '''
    cur = query_cursor(wh_name=wh_name, db_name=db_name, schema_name=schema_name)
    print(f'Snowflake Query ID: {cur.sfqid}')
    results = cur.execute(f'{sql_query}').fetchall()
    
    return pd.DataFrame(results, columns=col_names)

# Create Connection To Warehouse, Database and Schema

In [3]:
cur = query_cursor(WAREHOUSE, DATABASE, SCHEMA)

# Query Data

In [4]:
column_names = ['orderkey', 'custkey', 'orderstatus', 'totalprice', 'orderdate', 'orderpriority']

df = fetch_data(WAREHOUSE, DATABASE, SCHEMA, query1, column_names)

Snowflake Query ID: 01adb182-3201-dce7-0002-0ed20001903a


In [5]:
df

,orderkey,custkey,orderstatus,totalprice,orderdate,orderpriority
0,2400161,25882,O,158337.78,1996-12-29,4-NOT SPECIFIED
1,2400672,99451,O,175200.51,1997-12-21,1-URGENT
2,2400071,25478,F,7623.08,1994-07-02,1-URGENT
3,2400101,19960,O,223678.02,1996-02-25,5-LOW
4,2400102,37369,O,233587.79,1996-12-09,1-URGENT
...,...,...,...,...,...,...
1499995,1799846,41851,O,145278.77,1997-08-01,3-MEDIUM
1499996,1799877,139166,F,161819.05,1993-01-09,2-HIGH
1499997,1799878,7045,O,189946.79,1996-06-23,5-LOW
1499998,1799938,147031,O,167871.22,1997-03-27,3-MEDIUM
